In [33]:
"""
Read in anonymized data, output features matrix for modeling
"""
import pandas as pd
from fuzzywuzzy import process, fuzz
import re


# read in data
df = pd.read_csv('../data/CYSOanonymized.csv')

# set financial aid status to 0 if NaN

df['finaid'].fillna(0,inplace=True)

# parse race into different rows
df['race'].fillna('Other',inplace=True)
df['caucasian']=0
df['african_american']=0
df['native_american']=0
df['latino']=0
df['asian']=0
df['other_race']=0
df['race_num']=0

for index, row in df.iterrows():
    if re.search('cauc', str(row['race']), re.IGNORECASE):
        df.loc[index,'caucasian']=1
        df.loc[index,'race_num']=df.loc[index,'race_num']+1
    if re.search('african', str(row['race']), re.IGNORECASE):
        df.loc[index,'african_american']=1
        df.loc[index,'race_num']=df.loc[index,'race_num']+1
    if re.search('native', str(row['race']), re.IGNORECASE):
        df.loc[index,'caucasian']=1
        df.loc[index,'race_num']=df.loc[index,'race_num']+1
    if re.search('latino', str(row['race']), re.IGNORECASE):
        df.loc[index,'latino']=1
        df.loc[index,'race_num']=df.loc[index,'race_num']+1
    if re.search('asian', str(row['race']), re.IGNORECASE):
        if not re.search('caucasian', str(row['race']), re.IGNORECASE):
            df.loc[index,'asian']=1
            df.loc[index,'race_num']=df.loc[index,'race_num']+1
    if re.search('other', str(row['race']), re.IGNORECASE):
        df.loc[index,'other_race']=1
        df.loc[index,'race_num']=df.loc[index,'race_num']+1
    if re.search("`", str(row['race']), re.IGNORECASE):
        df.loc[index,'other_race']=1
        df.loc[index,'race_num']=df.loc[index,'race_num']+1

# compile gender
df['male']=0
df['female']=0

df.gender = df.gender.str.strip()
df.gender.replace('Male','M',inplace=True)
df.gender.replace('Female','F',inplace=True)

for index, row in df.iterrows():
    if row['gender']=='M':
        df.loc[index, 'male']=1
    if row['gender']=='F':
        df.loc[index,'female']=1

# compile major

df['music']=0

music_terms = ['performance','music','violin','songwriting','perfomance','bass','cello','viola','jazz']

for index, row in df.iterrows():
    for music_term in music_terms:
        if re.search(music_term,str(row['major']),re.IGNORECASE):
            df.loc[index,'music']=1
    

'''
map in and combine incomes
'''
#read in incomes and convert to numeric
income = pd.read_csv('../data/income_zip.csv')
df['homezip']=pd.to_numeric(df['homezip'], errors='coerce')

# merge with data structure
df = pd.merge(df,income,how='left',left_on='homezip',right_on='Zip')


# rename and reformat columns
df.rename(index=str, columns={"Mean": "mean_income", "Median": "median_income", \
                              "finaid" : "financial_aid", "race_num" : "multiracial"}, inplace=True)
df['mean_income'] = df['median_income'].str.replace(',','')
df['mean_income']=pd.to_numeric(df['mean_income'], errors='coerce')
df['median_income'] = df['median_income'].str.replace(',','')
df['median_income']=pd.to_numeric(df['median_income'], errors='coerce')
df['income_diff']=df['mean_income']-df['median_income']

'''
map in and combine college data
'''

# read in college data
uf = pd.read_csv('../data/colleges.csv')

# filter to relevant universal columns
uf = uf[['displayName','acceptance-rate','institutionalControl']]

# build columns for merging and saving
df['uni_close'] = 'none'
df['uni_save'] = df['uni']

# replace strings

# strip name function
def stripnames(dataframe,instring,outstring):
    dataframe.loc[df['uni'].str.replace(' ','').replace("'",'').replace('-','') \
                  ==instring.replace(' ','').replace("'",'').replace('-',''),'uni']=outstring
    return dataframe

# strip and modify names for matching
uf['displayName'] = uf['displayName'].fillna('none')
df['uni'] = df['uni'].fillna('none given')
df['uni'] = df['uni'].str.replace('Uof','University of')
df['uni'] = df['uni'].str.replace('U of','University of')
df['uni'] = df['uni'].str.replace('Jacobs School of Music','')
df = stripnames(df,'Indiana University','Indiana University--Bloomington')
df = stripnames(df,'NYU','New York University')
df = stripnames(df,'Oberlin','Oberlin College')
df = stripnames(df,'UIC','University of Illinois Chicago')
df = stripnames(df,'UIUC','University of Illinois--Urbana Champaign')
df = stripnames(df,'USC(So.Cal.)','University of Southern California')
df = stripnames(df,'UofMichigan','University of Michigan')
df = stripnames(df,'Peabody','Johns Hopkins University')
df = stripnames(df,'University of I','University of Illinois--Urbana Champaign')
df = stripnames(df,'NIU','Northern Illinois University')
df = stripnames(df,'U.ofMinnesotaCarlsonSchoolofBusiness','University of Minnesota')
df = stripnames(df,'NEC','New England Conservatory')
df = stripnames(df,'MIT','Massachussetts Institute of Technology')
df = stripnames(df,'Gap Year','none')
df = stripnames(df,'Year off','none')
df = stripnames(df,'The Juilliard School','Julliard School')
df = stripnames(df,'IUBloomington,JacobsSchoolofMusic','Indiana University--Bloomington')
df = stripnames(df,'IUJacobsSchoolofMusic','Indiana University--Bloomington')
df = stripnames(df,'HarvardCollege','Harvard University')
df = stripnames(df,'Undecided','none')
df = stripnames(df,'NotgoingtoCollege','none')
df = stripnames(df,'IndianaUniversityJacobsSchoolofMusic','Indiana University--Bloomington')
df = stripnames(df,'IndianaUniversity,JacobsSchoolofMusic','Indiana University--Bloomington')
df = stripnames(df,'IndianaUniversity(JacobsSchoolofMusic)','Indiana University--Bloomington')
df = stripnames(df,'JacobsSchoolofMusic,atIndianaUniversity','Indiana University--Bloomington')
df = stripnames(df,'ClarendonHillsMiddleSchool','none')
df = stripnames(df,'IU','Indiana University--Bloomington')
df = stripnames(df,'LawrenceConservatory','Lawrence University')
df = stripnames(df,'IndianaUniversity','Indiana University--Bloomington')
df = stripnames(df,'WesternIllinoisUniversity','WesternIllinois')
df = stripnames(df,'UniversityofIllinois','University of Illinois--Urbana Champaign')
df = stripnames(df,',atIndianaUniversity','Indiana University--Bloomington')

# match name to closest
for index, row in df.iterrows():
    df.loc[index, 'uni_close']=process.extractOne(row['uni'],uf['displayName'])[0]


# merge university data into full data structure
df = pd.merge(df,uf,how='left',left_on='uni_close',right_on='displayName')



# encode university types
df['uni_public'] = 0
# df.loc(df['institutionalControl']=='private','institutionalControl')=0
# df.loc(df['institutionalControl']=='none','institutionalControl')=0
df.loc[df['institutionalControl']=='public','uni_public']=1

# drop spurious columns and rename others
df.drop(['Unnamed: 0','Zip','institutionalControl','displayName','uni_save','uni'],axis=1, inplace=True)
df.rename(index=str, columns={"uni_close": "uni"}, inplace=True)

# save full data structure
df.to_csv('../data/exploring_data.csv')

# # drop all data not used in modeling - comment in when prepping data
# df.drop(['Zip', 'Pop', 'Unnamed: 0','instrument','race','gender','s_ensemble',\
#         'm_employ','m_job','d_employ','school','s_address','uni','pt',\
#         's_state','major','homezip','s_zip','studentID','mean_income',
#         'gradyear','female','native_american','income_diff'], axis=1, inplace=True)


# for non-logistic regression classifiers
df = df[['music','male','female','caucasian','african_american',
         'latino', 'asian','other_race','multiracial','financial_aid',
         'median_income','acceptance-rate','uni_public']]

# save fitting data structure
df.to_csv('../data/fitting_data.csv')

In [30]:
df.instrument.unique()

array(['Flute', 'Oboe', 'Clarinet', 'Bassoon', 'French Horn', 'Trumpet',
       'Trombone', 'Tuba', 'Percussion', 'Harp', 'Violin', 'Viola',
       'Cello', 'Bass', 'Bass Trombone', 'Piano', 'Saxophone', 'Steelpan',
       'Percussion/Drums', 'Alto Saxophone', 'Drum Set', 'Steel Pan',
       'Clarinet,Alto Saxophone'], dtype=object)

In [34]:
df.columns

Index(['music', 'male', 'female', 'caucasian', 'african_american', 'latino',
       'asian', 'other_race', 'multiracial', 'financial_aid', 'median_income',
       'acceptance-rate', 'uni_public'],
      dtype='object')